因为LeNet被SVM取代，当初也没有像GPU那样大量普及操作，所以有大量参数的卷积神经网络训练在当时是很难完成的。

当时手工提取特征的方法总结如下：

1. 获取图像数据集
2. 使用已有的特征提取函数生成图像特征
3. 使用机器学习模型对图像的特征进行分类

# 学习特征的表示

为表征足够复杂的输入，特征本身应该分级表示。

多层神经网络可能学习得到数据的多级表征，并逐级表示越来越抽象的概念或模式。

不过，试图学习数据的逐级表征，还是存在很多阻碍的因素的。

## 阻碍一：数据

公开数据集的数据量太小，直至大数据时代，才有了合适的数据。

## 阻碍二：硬件

深度学习对计算资源要求很高。

# AlexNet

8层卷积神经网络，赢得了ImageNet比赛，首次证明了学习到的特征可以超越手工设计的特征。

8层：5层卷积、2层全连接隐藏层，1层全连接输出层。

## 设计
### 卷积层
卷积通道数是LeNet的10倍

第一层：11 * 11的卷积窗口，因为图像数据较大

第二层：5 * 5 

其余卷积层：3 * 3

最后一层卷积层的两个输出个数为4096的全连接层。参数大概都得占用1GB。

### 池化层
在1、2、5卷积层之后使用了3 * 3的最大池化窗口，步幅为2。

### 激活函数
ReLU函数取代了Sigmoid函数

1. 因为Sigmoid函数可能在正区间得到几乎为0的梯度
2. Sigmoid函数存在求幂运算

### Dropout
控制全连接层的模型复杂度

### 缓解过拟合
引入了大量的图像增广、翻转、裁剪、颜色变化，缓解过拟合

# 实现简化的AlexNet

In [1]:
import time
import torch
import torchvision
from torch import nn, optim
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 前两个卷积层不用池化见效输入和输出的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
        # 全连接层，Dropout
        self.fc = nn.Sequential(
            # 参数TODO，Dropout
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [3]:
net = AlexNet()
print(net)

AlexNet(
  (conv): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6400, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (

In [4]:
batch_size = 128
root = '~/Datasets'
train_iter, test_iter = d2l.load_data_fashion_mnist2(batch_size, resize=224, root=root)

In [5]:
# mac不能写num_worker
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_cuda_cpu(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cpu


KeyboardInterrupt: 

诶。。。我这乞丐机train不出来，算了算了